In [1]:
pip install transformers peft datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import login
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `aml v3` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

### google flan-t5-base

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset
from google.colab import files
import json

# **1. 데이터 로드**
uploaded = files.upload()
data_path = list(uploaded.keys())[0]

def load_instruction_data(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    dataset = Dataset.from_list(data)
    return dataset

dataset = load_instruction_data(data_path)

# **2. 데이터셋 분리**
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# **3. 모델 및 토크나이저 로드**
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# **4. 데이터 전처리**
def preprocess_function(examples):
    inputs = [
        f"Instruction: {instruction}\nInput: {input_text}\nOutput:"
        for instruction, input_text in zip(examples["instruction"], examples["input"])
    ]
    labels = examples["output"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(labels, max_length=512, truncation=True, padding="max_length")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# **5. TrainingArguments 설정**
training_args = TrainingArguments(
    output_dir="./flan-t5-base-movie-recommendation",
    eval_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=500,
    logging_steps=100,
    fp16=True,
    push_to_hub=False,
    report_to="none"
)

# **6. Trainer 초기화**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
)

# **7. 모델 훈련**
trainer.train()

# **8. 모델 저장**
model.save_pretrained("./flan-t5-base-movie-recommendation")
tokenizer.save_pretrained("./flan-t5-base-movie-recommendation")


Saving generated_data.json to generated_data.json


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,0.000000
200,0.000000
300,0.000000
400,0.000000
500,0.000000
600,0.000000
700,0.000000
800,0.000000
900,0.000000
1000,0.000000


('./flan-t5-base-movie-recommendation/tokenizer_config.json',
 './flan-t5-base-movie-recommendation/special_tokens_map.json',
 './flan-t5-base-movie-recommendation/spiece.model',
 './flan-t5-base-movie-recommendation/added_tokens.json',
 './flan-t5-base-movie-recommendation/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 저장 디렉토리
output_dir = "./movie-recommendation-model"

# 모델과 토크나이저 저장
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to ./movie-recommendation-model


In [ ]:
import shutil

# 모델 디렉토리 경로
output_dir = "./movie-recommendation-model"

# 압축 파일 경로
zip_file = "movie-recommendation-model.zip"

# 디렉토리 압축
shutil.make_archive("movie-recommendation-model", 'zip', output_dir)
print(f"Model compressed to {zip_file}")


Model compressed to movie-recommendation-model.zip


In [ ]:
from google.colab import files

# 압축 파일 다운로드
files.download("movie-recommendation-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pwd  # 현재 디렉토리 확인
!ls  # 파일 목록 확인


/content
flan-t5-base-movie-recommendation  movie-recommendation-model	   sample_data
generated_data.json		   movie-recommendation-model.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Google Drive에 저장
!cp movie-recommendation-model.zip /content/drive/MyDrive/
print("File saved to Google Drive!")


Mounted at /content/drive
File saved to Google Drive!


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델과 토크나이저 로드
model_dir = "./movie-recommendation-model"  # 훈련된 모델 저장 경로
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [ ]:
instruction = "Considering the user's movie history, what is the best next movie to watch?"
input_text = "Shawshank Redemption, The (1994), Fight Club (1999), Face/Off (1997), Paycheck (2003), Mission: Impossible II (2000), Broken Arrow (1996), Windtalkers (2002), Hard-Boiled (Lat sau san taam) (1992), Killer, The (Die xue shuang xiong) (1989), Hard Target (1993), Red Cliff (Chi bi) (2008), Red Cliff Part II (Chi Bi Xia: Jue Zhan Tian Xia) (2009), Better Tomorrow, A (Ying hung boon sik) (1986), Bullet in the Head (1990), Better Tomorrow II, A (Ying hung boon sik II) (1987), Once a Thief (Zong heng si hai) (1991), Last Hurrah for Chivalry (Hao xia) (1979), Slumdog Millionaire (2008), Trainspotting (1996), 28 Days Later (2002), 127 Hours (2010), Sunshine (2007), Beach, The (2000), Trance (2013), Shallow Grave (1994), Millions (2004), Life Less Ordinary, A (1997), Departed, The (2006), Wolf of Wall Street, The (2013), Shutter Island (2010), Goodfellas (1990), Taxi Driver (1976), Gangs of New York (2002), Aviator, The (2004), Casino (1995), Hugo (2011), Raging Bull (1980), Cape Fear (1991), After Hours (1985), King of Comedy, The (1983), Mean Streets (1973), Bringing Out the Dead (1999), Last Temptation of Christ, The (1988), Color of Money, The (1986), Age of Innocence, The (1993), Kundun (1997), Alice Doesn't Live Here Anymore (1974), Hitchcock/Truffaut (2015), New York, New York (1977), Boxcar Bertha (1972), Who's That Knocking at My Door? (1967), Rush Hour (1998), Robin-B-Hood (Bo bui gai wak) (2006), Miracles - Mr. Canton and Lady Rose (1989), Kung Fu Panda (2008), Rush Hour 2 (2001), Shanghai Noon (2000), Kung Fu Panda 2 (2011), Kung Fu Panda 3 (2016), Shanghai Knights (2003), Rush Hour 3 (2007), Forbidden Kingdom, The (2008), Rumble in the Bronx (Hont faan kui) (1995), Enter the Dragon (1973), Around the World in 80 Days (2004), Legend of Drunken Master, The (Jui kuen II) (1994), Drunken Master (Jui kuen) (1978), Medallion, The (2003), Who Am I? (Wo shi shei) (1998), Police Story (Ging chaat goo si) (1985), First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: Ji gaan daan yam mo) (1996), Mr. Nice Guy (Yat goh ho yan) (1997), Armour of God (Long xiong hu di) (1987), Police Story 2 (Ging chaat goo si juk jaap) (1988), New Police Story (Xin jing cha gu shi) (2004), Project A ('A' gai waak) (1983), Armour of God II: Operation Condor (Operation Condor) (Fei ying gai wak) (1991), Little Big Soldier (Da bing xiao jiang) (2010), Snake in the Eagle's Shadow (Se ying diu sau) (1978), Accidental Spy, The (Dak miu mai shing) (2001), Project A 2 ('A' gai wak juk jap) (1987), Wheels on Meals (Kuai can che) (1984), City Hunter (Sing si lip yan) (1993), Winners and Sinners (Qi mou miao ji: Wu fu xing) (1983), King of Comedy (Hei kek ji wong) (1999), Dragon Lord (a.k.a. Dragon Strike) (Long Xiao Ye) (1982), Fearless Hyena, The (Xiao quan guai zhao) (1979), My Lucky Stars (Fuk sing go jiu) (1985), Crime Story (Zhong an zu) (1993), Gen-X Cops (1999), The Raid: Redemption (2011), The Raid 2: Berandal (2014), V/H/S/2 (2013), Merantau (2009), Before Sunrise (1995), World's End, The (2013), Shaun of the Dead (2004), Hot Fuzz (2007), Scott Pilgrim vs. the World (2010), Avengers: Age of Ultron (2015), Avengers, The (2012), Cabin in the Woods, The (2012), Spider-Man 2 (2004), Spider-Man (2002), Spider-Man 3 (2007), Evil Dead, The (1981), Army of Darkness (1993), Evil Dead II (Dead by Dawn) (1987), Drag Me to Hell (2009), Simple Plan, A (1998), Gift, The (2000), Quick and the Dead, The (1995), Darkman (1990), Maniac Cop (1988), Maniac Cop 2 (1990), Looper (2012), Brick (2005), Brothers Bloom, The (2008), 40-Year-Old Virgin, The (2005), Anchorman: The Legend of Ron Burgundy (2004), Knocked Up (2007), Trainwreck (2015), This Is 40 (2012), Funny People (2009), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Fifth Element, The (1997), Lucy (2014), Femme Nikita, La (Nikita) (1990), Family, The (2013), School of Rock (2003), Boyhood (2014), Spy Kids (2001), Scanner Darkly, A (2006), Dazed and Confused (1993), Waking Life (2001), Everybody Wants Some (2016), Bernie (2011), Slacker (1991), Bad News Bears (2005), SubUrbia (1997), Bad News Bears, The (1976), Back to the Future (1985), Godfather: Part II, The (1974), Toy Story (1995), City of God (Cidade de Deus) (2002), Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964), Pan's Labyrinth (Laberinto del fauno, El) (2006), Prestige, The (2006), Little Miss Sunshine (2006), Thing, The (1982), Toy Story 2 (1999), District 9 (2009), Tucker & Dale vs Evil (2010), Vertigo (1958), Fargo (1996), Apocalypse Now (1979), Citizen Kane (1941), Short Term 12 (2013), Reservoir Dogs (1992), Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il) (1966), Dawn of the Dead (1978), Chronicle (2012), Superbad (2007), Monsters, Inc. (2001), Argo (2012), Her (2013), Full Metal Jacket (1987), E.T. the Extra-Terrestrial (1982), Terminator 2: Judgment Day (1991), I Saw the Devil (Akmareul boatda) (2010), Ghostbusters (a.k.a. Ghost Busters) (1984), Mean Girls (2004), The Martian (2015), Dog Day Afternoon (1975), Star Wars: Episode IV - A New Hope (1977), Shining, The (1980), Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001), Truman Show, The (1998), Dead Alive (Braindead) (1992), Exorcist, The (1973), Halloween (1978), Escape from New York (1981), They Live (1988), Big Trouble in Little China (1986), Escape from L.A. (1996), Christine (1983), In the Mouth of Madness (1995), Starman (1984), Ghosts of Mars (2001), Fog, The (1980), Vampires (1998), Assault on Precinct 13 (1976), Prince of Darkness (1987), Village of the Damned (1995), Memoirs of an Invisible Man (1992), Cigarette Burns (2005), Troll 2 (1990), Undisputed II: Last Man Standing (2006), Ninja: Shadow of a Tear (2013), Shepherd: Border Patrol, The (2008), Undisputed III: Redemption (2010), Close Range (2015), Ninja (2009), Rubber (2010), Reality (2014), Wrong (2012), Wrong Cops (2013), Eternal Sunshine of the Spotless Mind (2004), Green Hornet, The (2011), Science of Sleep, The (La science des rêves) (2006), Be Kind Rewind (2008), Tokyo! (2008), Mind Game (2004), Monty Python and the Holy Grail (1975), Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Brazil (1985), Fear and Loathing in Las Vegas (1998), Jupiter Ascending (2015), Monty Python's Life of Brian (1979), Monty Python's The Meaning of Life (1983), Brothers Grimm, The (2005), Imaginarium of Doctor Parnassus, The (2009), Fisher King, The (1991), Zero Theorem, The (2013), Adventures of Baron Munchausen, The (1988), Tideland (2005), Kill List (2011), Sightseers (2012), Field in England, A (2013), High Rise (2015), Down Terrace (2009), Aliens (1986), Terminator, The (1984), Abyss, The (1989), Avatar (2009), True Lies (1994), Titanic (1997), Five Deadly Venoms (1978), Crippled Avengers (Can que) (Return of the 5 Deadly Venoms) (1981), Five Element Ninjas (1982), Shaolin Temple (Shao Lin si) (1976), Invincible Shaolin (1978), The Magnificent Ruffians (1979), Masked Avengers (1981), Return of the One-Armed Swordsman (1969), New One-Armed Swordsman, The (Xin du bi dao) (1971), One-Armed Swordsman, The (Dubei dao) (1967), Dumb & Dumber (Dumb and Dumber) (1994), There's Something About Mary (1998), Shallow Hal (2001), Me, Myself & Irene (2000), Dumb and Dumber To (2014), Kingpin (1996), Hall Pass (2011), Movie 43 (2013), Stuck on You (2003), Fever Pitch (2005), Heartbreak Kid, The (2007), Big Lebowski, The (1998), Barton Fink (1991), No Country for Old Men (2007), Hail, Caesar! (2016), True Grit (2010), O Brother, Where Art Thou? (2000), Burn After Reading (2008), Serious Man, A (2009), Inside Llewyn Davis (2013), Miller's Crossing (1990), Ladykillers, The (2004), Intolerable Cruelty (2003), Blood Simple (1984), Hudsucker Proxy, The (1994), Man Who Wasn't There, The (2001), Raising Arizona (1987), Broken Flowers (2005), Dead Man (1995), Sling Blade (1996), Only Lovers Left Alive (2013), Ghost Dog: The Way of the Samurai (1999), Coffee and Cigarettes (2003), Down by Law (1986), Night on Earth (1991), Stranger Than Paradise (1984), Mystery Train (1989), Limits of Control, The (2009), Smoke (1995), Star Wars: Episode III - Revenge of the Sith (2005), Star Wars: Episode II - Attack of the Clones (2002), Star Wars: Episode V - The Empire Strikes Back (1980), Star Wars: Episode VI - Return of the Jedi (1983), Star Wars: Episode VII - The Force Awakens (2015), Green Mile, The (1999), Let the Right One In (Låt den rätte komma in) (2008), Ace Ventura: Pet Detective (1994), Mask, The (1994), Kick-Ass 2 (2013), Liar Liar (1997), Anchorman 2: The Legend Continues (2013), Bruce Almighty (2003), Batman Forever (1995), Cable Guy, The (1996), How the Grinch Stole Christmas (a.k.a. The Grinch) (2000), Lemony Snicket's A Series of Unfortunate Events (2004), Man on the Moon (1999), My Life as a Dog (Mitt liv som hund) (1985), My Dinner with André (1981), Bicycle Thieves (a.k.a. The Bicycle Thief) (a.k.a. The Bicycle Thieves) (Ladri di biciclette) (1948), Once Upon a Time in the West (C'era una volta il West) (1968), Fistful of Dollars, A (Per un pugno di dollari) (1964), For a Few Dollars More (Per qualche dollaro in più) (1965), Once Upon a Time in America (1984), Duck, You Sucker (1971), Suspiria (1977), Deep Red (Profondo rosso) (1975), Opera (1987), Enter the Void (2009), Irreversible (Irréversible) (2002), Dobermann (1997), Battles Without Honor & Humanity (Jingi naki tatakai) (1973), Battle Royale (Batoru rowaiaru) (2000), Sympathy for the Underdog (1971), Branded to Kill (Koroshi no rakuin) (1967), Take Aim at the Police Van (Sono gosôsha wo nerae: 'Jûsangô taihisen' yori) (1960), Tokyo Drifter (Tôkyô nagaremono) (1966), Youth of the Beast (Yaju no seishun) (1963), Tattooed Life (Irezumi ichidai) (1965), Old Boy (2003), Lady Vengeance (Sympathy for Lady Vengeance) (Chinjeolhan geumjassi) (2005), Stoker (2013), Sympathy for Mr. Vengeance (Boksuneun naui geot) (2002), Thirst (Bakjwi) (2009), Joint Security Area (Gongdong gyeongbi guyeok JSA) (2000), I'm a Cyborg, But That's OK (Saibogujiman kwenchana) (2006), Host, The (Gwoemul) (2006), Memories of Murder (Salinui chueok) (2003), Mother (Madeo) (2009), Barking Dogs Never Bite (Flandersui gae) (2000), Snowpiercer (2013), Holy Motors (2012), Boy Meets Girl (1984), Kiss Kiss Bang Bang (2005), The Nice Guys (2016), Predator (1987), As Good as It Gets (1997), Iron Man 3 (2013), RoboCop 3 (1993), Dead Heat (1988), Lethal Weapon (1987), Long Kiss Goodnight, The (1996), Last Boy Scout, The (1991), 13 Assassins (Jûsan-nin no shikaku) (2010), Last Life in the Universe (Ruang rak noi nid mahasan) (2003), Visitor Q (Bizita Q) (2001), Happiness of the Katakuris, The (Katakuri-ke no kôfuku) (2001), Crows Zero (Kurôzu zero) (2007), Bird People in China, The (Chûgoku no chôjin) (1998), Dead or Alive 2: Tôbôsha (2000), Dead or Alive: Hanzaisha (1999), Sukiyaki Western Django (2008), Hostel (2005), Audition (Ôdishon) (1999), Ichi the Killer (Koroshiya 1) (2001), Gozu (Gokudô kyôfu dai-gekijô: Gozu) (2003), Zebraman (2004), Great Yokai War, The (Yôkai daisensô) (2005), Deadly Outlaw: Rekka (a.k.a. Violent Fire) (Jitsuroku Andô Noboru kyôdô-den: Rekka) (2002), City of Lost Souls, The (Hyôryuu-gai) (2000), Exte: Hair Extensions (2007), Cold Fish (Tsumetai nettaigyo) (2010), Suicide Club (Jisatsu saakuru) (2001), Himizu (2011), Strange Circus (Kimyô na sâkasu) (2005), Love Exposure (Ai No Mukidashi) (2008), Why Don't You Play In Hell? (Jigoku de naze warui) (2013), Tokyo Tribe (2014), This Is England (2006), Dead Man's Shoes (2004), Room for Romeo Brass, A (1999), Expendables, The (2010), Bloodmoon (1997), Total Recall (1990), RoboCop (1987), Starship Troopers (1997), Hollow Man (2000), Pulp Fiction (1994), Silence of the Lambs, The (1991), Jurassic Park (1993), Matrix, The (1999), Schindler's List (1993), Braveheart (1995), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Fugitive, The (1993), Independence Day (a.k.a. ID4) (1996), American Beauty (1999), Godfather, The (1972), Batman (1989), Lord of the Rings: The Fellowship of the Ring, The (2001), Seven (a.k.a. Se7en) (1995), Saving Private Ryan (1998), Lord of the Rings: The Two Towers, The (2002), Sixth Sense, The (1999), Lord of the Rings: The Return of the King, The (2003), Speed (1994), Dog Soldiers (2002), Bittersweet Life, A (Dalkomhan insaeng) (2005), Gladiator (2000), Shrek (2001), Memento (2000), Dark Knight, The (2008), Pirates of the Caribbean: The Curse of the Black Pearl (2003), X-Men (2000), Inception (2010), Finding Nemo (2003), Kill Bill: Vol. 1 (2003), Usual Suspects, The (1995), Forrest Gump (1994), Flesh & Blood (1985), Somers Town (2008), Hazard (2005), Minority Report (2002), Ocean's Eleven (2001), Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000), Incredibles, The (2004), Batman Begins (2005), Bourne Identity, The (2002), Kill Bill: Vol. 2 (2004), Donnie Darko (2001), Catch Me If You Can (2002), Cast Away (2000), Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001), Snatch (2000), V for Vendetta (2006), WALL·E (2008), Iron Man (2008), Matrix Reloaded, The (2003), Almost Famous (2000), Up (2009), Sin City (2005), Requiem for a Dream (2000), Spirited Away (Sen to Chihiro no kamikakushi) (2001), Harry Potter and the Prisoner of Azkaban (2004), Harry Potter and the Chamber of Secrets (2002), Patriot, The (2000), Django Unchained (2012), Royal Tenenbaums, The (2001), Dark Knight Rises, The (2012), Bridget Jones's Diary (2001), I, Robot (2004), Last Samurai, The (2003), Pirates of the Caribbean: Dead Man's Chest (2006), Moulin Rouge (2001), Star Trek (2009), Bowling for Columbine (2002), Hangover, The (2009), A.I. Artificial Intelligence (2001), Children of Men (2006), Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (2002), Signs (2002), Black Hawk Down (2001), Italian Job, The (2003), Others, The (2001), Lost in Translation (2003), Bourne Supremacy, The (2004), Traffic (2000), Inglourious Basterds (2009), Bourne Ultimatum, The (2007), High Fidelity (2000), Interstellar (2014), Big Fish (2003), Juno (2007), King's Speech, The (2010), Love Actually (2003), Sherlock Holmes (2009), I Am Legend (2007), Mulholland Drive (2001), American Psycho (2000), Garden State (2004), Collateral (2004), Inside Out (2015), Thank You for Smoking (2006), Road to Perdition (2002), Gone Girl (2014), Remember the Titans (2000), Billy Elliot (2000), Grand Budapest Hotel, The (2014), Hannibal (2001), Into the Wild (2007), Moon (2009), Sideways (2004), In Bruges (2008), Panic Room (2002), Lives of Others, The (Das leben der Anderen) (2006), Stranger than Fiction (2006), What Lies Beneath (2000), Life Aquatic with Steve Zissou, The (2004), Hitch (2005), Constantine (2005), American Gangster (2007), Man on Fire (2004), Hurt Locker, The (2008), Ghost World (2001), Phone Booth (2002), Silver Linings Playbook (2012), Drive (2011), Girl with the Dragon Tattoo, The (2011), Village, The (2004), Zodiac (2007), The Count of Monte Cristo (2002), Punch-Drunk Love (2002), 25th Hour (2002), Boiler Room (2000), Birdman: Or (The Unexpected Virtue of Ignorance) (2014), Secretary (2002), History of Violence, A (2005), Dallas Buyers Club (2013), House of Flying Daggers (Shi mian mai fu) (2004), Nightcrawler (2014), Bad Santa (2003), World War Z (2013), Crazy, Stupid, Love. (2011), Click (2006), 50/50 (2011), Fountain, The (2006), American Splendor (2003), Gone Baby Gone (2007), Town, The (2010), Fighter, The (2010), Primer (2004), Jarhead (2005), X2: X-Men United (2003), Matrix Revolutions, The (2003), Terminator 3: Rise of the Machines (2003), Guardians of the Galaxy (2014), Austin Powers in Goldmember (2002), Vanilla Sky (2001), Planet of the Apes (2001), The Hunger Games (2012), Watchmen (2009), X-Men: The Last Stand (2006), Equilibrium (2002), Mad Max: Fury Road (2015), War of the Worlds (2005), Iron Man 2 (2010), Day After Tomorrow, The (2004), Pitch Black (2000), Transformers (2007), X-Men: First Class (2011), Jurassic Park III (2001), League of Extraordinary Gentlemen, The (a.k.a. LXG) (2003), Captain America: The Winter Soldier (2014), Star Trek Into Darkness (2013), X-Men Origins: Wolverine (2009), 6th Day, The (2000), Evolution (2001), Prometheus (2012), Cloverfield (2008), 28 Weeks Later (2007), Chronicles of Riddick, The (2004), AVP: Alien vs. Predator (2004), 2012 (2009), Superman Returns (2006), Incredible Hulk, The (2008), Book of Eli, The (2010), The Hunger Games: Mockingjay - Part 1 (2014), Cloud Atlas (2012), Tron: Legacy (2010), Nutty Professor II: The Klumps (2000), Dawn of the Planet of the Apes (2014), Pacific Rim (2013), Resident Evil: Apocalypse (2004), About Time (2013), Underworld: Evolution (2006), Cowboy Bebop: The Movie (Cowboy Bebop: Tengoku no Tobira) (2001), Idiocracy (2006), Maze Runner, The (2014), Captain America: Civil War (2016), Blade: Trinity (2004), Arrival (2016), Mist, The (2007), One, The (2001), 9 (2009), Ender's Game (2013), Predestination (2014), Dreamcatcher (2003), Cowboys & Aliens (2011), John Carter (2012), AVPR: Aliens vs. Predator - Requiem (2007), Day the Earth Stood Still, The (2008), Chappie (2015), Melancholia (2011), Happening, The (2008), Godzilla (2014), Flintstones in Viva Rock Vegas, The (2000), Pandorum (2009), Doctor Strange (2016), Vampire Hunter D: Bloodlust (Banpaia hantâ D) (2000), Devil's Backbone, The (Espinazo del diablo, El) (2001), Safety Not Guaranteed (2012), 10 Cloverfield Lane (2016), Ultraviolet (2006), Terminator Genisys (2015), Riddick (2013), Predators (2010), X-Men: Apocalypse (2016), Charlie's Angels (2000), Casino Royale (2006), 300 (2007), Unbreakable (2000), Miss Congeniality (2000), Gone in 60 Seconds (2000), Mr. & Mrs. Smith (2005), Blood Diamond (2006), Mummy Returns, The (2001), Training Day (2001), Pirates of the Caribbean: At World's End (2007), Taken (2008), Kick-Ass (2010), Lara Croft: Tomb Raider (2001), Fast and the Furious, The (2001), Edge of Tomorrow (2014), Hero (Ying xiong) (2002), Deadpool (2016), Hellboy (2004), Big Hero 6 (2014), Rise of the Planet of the Apes (2011), Live Free or Die Hard (2007), Hancock (2008), Underworld (2003), Team America: World Police (2004), Quantum of Solace (2008), Munich (2005), Life of Pi (2012), xXx (2002), Tropic Thunder (2008), Kingsman: The Secret Service (2015), Dawn of the Dead (2004), Van Helsing (2004), Charlie's Angels: Full Throttle (2003), Mission: Impossible III (2006), Transporter, The (2002), Grindhouse (2007), Mission: Impossible - Ghost Protocol (2011), 2 Fast 2 Furious (Fast and the Furious 2, The) (2003), Romeo Must Die (2000), Red (2010), Pirates of the Caribbean: On Stranger Tides (2011), Once Upon a Time in Mexico (2003), Jurassic World (2015), Vertical Limit (2000), Shooter (2007), Ant-Man (2015), Terminator Salvation (2009), Oblivion (2013), Sky Captain and the World of Tomorrow (2004), Kingdom of Heaven (2005), Transformers: Revenge of the Fallen (2009), Elysium (2013), Men in Black III (M.III.B.) (M.I.B.³) (2012), Man of Steel (2013), Reign of Fire (2002), Thor: The Dark World (2013), Planet Terror (2007), Behind Enemy Lines (2001), Rush (2013), Ghost Rider (2007), A-Team, The (2010), Bourne Legacy, The (2012), The Hunger Games: Mockingjay - Part 2 (2015), Ip Man (2008), Transporter 2 (2005), 22 Jump Street (2014), Sicario (2015), Resident Evil: Extinction (2007), Transformers: Dark of the Moon (2011), Adventures of Tintin, The (2011), Machete (2010), 16 Blocks (2006), Wolverine, The (2013), Punisher, The (2004), Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006), Other Guys, The (2010), Hanna (2011), Knowing (2009), The Man from U.N.C.L.E. (2015), Total Recall (2012), This Is the End (2013), Encounters at the End of the World (2008), Logan (2017), Fast & Furious (Fast and the Furious 4, The) (2009), Fast Five (Fast and the Furious 5, The) (2011), Fast & Furious 6 (Fast and the Furious 6, The) (2013), Furious 7 (2015), The Fate of the Furious (2017), Blind Fury (1989), Chicken Run (2000), Ratatouille (2007), Toy Story 3 (2010), How to Train Your Dragon (2010), Simpsons Movie, The (2007), Final Fantasy: The Spirits Within (2001), Despicable Me (2010), Emperor's New Groove, The (2000), Cars (2006), Corpse Bride (2005), Titan A.E. (2000), Lilo & Stitch (2002), Wallace & Gromit in The Curse of the Were-Rabbit (2005), Coraline (2009), Zootopia (2016), Fantastic Mr. Fox (2009), The Lego Movie (2014), Wreck-It Ralph (2012), Cloudy with a Chance of Meatballs (2009), Mary and Max (2009), Beowulf (2007), Finding Dory (2016), Bug's Life, A (1998), Brave (2012), Monsters University (2013), The Good Dinosaur (2015), For the Birds (2000), Geri's Game (1997), One Man Band (2005), Lifted (2006), Cars 2 (2011), Ghost in the Shell (Kôkaku kidôtai) (1995), Akira (1988), Harry Potter and the Goblet of Fire (2005), Harry Potter and the Order of the Phoenix (2007), Charlie and the Chocolate Factory (2005), Harry Potter and the Half-Blood Prince (2009), Harry Potter and the Deathly Hallows: Part 1 (2010), Harry Potter and the Deathly Hallows: Part 2 (2011), Road Trip (2000), Hitchhiker's Guide to the Galaxy, The (2005), Hobbit: An Unexpected Journey, The (2012), Master and Commander: The Far Side of the World (2003), Skyfall (2012), King Kong (2005), Knight's Tale, A (2001), Thor (2011), The Hunger Games: Catching Fire (2013), Indiana Jones and the Kingdom of the Crystal Skull (2008), X-Men: Days of Future Past (2014), The Revenant (2015), Rat Race (2001), Bad Boys II (2003), Hellboy II: The Golden Army (2008), Jumper (2008), How to Train Your Dragon 2 (2014), The Scorpion King (2002), Brotherhood of the Wolf (Pacte des loups, Le) (2001), Joe Dirt (2001), Descent, The (2005), Paul (2011), Dungeons & Dragons (2000), Secret Life of Walter Mitty, The (2013), Spectre (2015), Hot Tub Time Machine (2010), The Amazing Spider-Man 2 (2014), Rundown, The (2003), Harold & Kumar Escape from Guantanamo Bay (2008), Batman v Superman: Dawn of Justice (2016), Expendables 2, The (2012), 3000 Miles to Graceland (2001), Way of the Gun, The (2000), Doom (2005), Cellular (2004), Spy Kids 2: The Island of Lost Dreams (2002), Art of War, The (2000), Transporter 3 (2008), Jimmy Neutron: Boy Genius (2001), Resident Evil: Afterlife (2010), Green Zone (2010), Tomorrowland (2015), Star Trek Beyond (2016), Proposition, The (2005), The Jungle Book (2016), Lone Ranger, The (2013), Meet the Parents (2000), Best in Show (2000), Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan (2006), Scary Movie (2000), About a Boy (2002), Zombieland (2009), Adaptation (2002), American Pie 2 (2001), (500) Days of Summer (2009), Zoolander (2001), Wedding Crashers (2005), 50 First Dates (2004), Napoleon Dynamite (2004), Bring It On (2000), Family Man, The (2000), Anger Management (2003), Nurse Betty (2000), Coyote Ugly (2000), Bedazzled (2000), Kung Fu Hustle (Gong fu) (2004), Harold and Kumar Go to White Castle (2004), Pineapple Express (2008), Scary Movie 2 (2001), Big Short, The (2015), Darjeeling Limited, The (2007), Station Agent, The (2003), Ted (2012), Along Came Polly (2004), Horrible Bosses (2011), Super Troopers (2001), Clerks II (2006), Talladega Nights: The Ballad of Ricky Bobby (2006), Confessions of a Dangerous Mind (2002), I Love You, Man (2009), Mighty Wind, A (2003), Big Momma's House (2000), Zack and Miri Make a Porno (2008), Blades of Glory (2007), Shaolin Soccer (Siu lam juk kau) (2001), Spanglish (2004), Lars and the Real Girl (2007), Igby Goes Down (2002), Rango (2011), Orange County (2002), Saved! (2004), National Lampoon's Van Wilder (2002), Descendants, The (2011), Squid and the Whale, The (2005), Brüno (Bruno) (2009), Dr. Dolittle 2 (2001), Bubba Ho-tep (2002), Seven Psychopaths (2012), Men Who Stare at Goats, The (2009), Vacation (2015), Lucky Number Slevin (2006), Saw (2004), Inside Man (2006), Insomnia (2002), Swordfish (2001), Red Dragon (2002), Sherlock Holmes: A Game of Shadows (2011), Daredevil (2003), Spy Game (2001), Matchstick Men (2003), Shaft (2000), Score, The (2001), Prisoners (2013), The Hateful Eight (2015), American Hustle (2013), S.W.A.T. (2003), Crank (2006), Sexy Beast (2000), Frailty (2001), Bank Job, The (2008), Skulls, The (2000), Infernal Affairs (Mou gaan dou) (2002), RocknRolla (2008), Snakes on a Plane (2006), United 93 (2006), Shoot 'Em Up (2007), Kiss of the Dragon (2001), Heartbreakers (2001), Narc (2002), Man on Wire (2008), Spy (2015), Equalizer, The (2014), Smokin' Aces (2006), Unleashed (Danny the Dog) (2005), Elite Squad (Tropa de Elite) (2007), Miami Vice (2006), Hostage (2005), Suicide Squad (2016), End of Watch (2012), Four Brothers (2005), Mechanic, The (2011), Side Effects (2013), Place Beyond the Pines, The (2012), Red 2 (2013), Domino (2005), Now You See Me 2 (2016), xXx: State of the Union (2005), Lookout, The (2007), Devil's Rejects, The (2005), Sin City: A Dame to Kill For (2014), Spartan (2004), Four Lions (2010), Headhunters (Hodejegerne) (2011), Bully (2001), American, The (2010), Hell or High Water (2016), Tell No One (Ne le dis à personne) (2006), Elf (2003), Stardust (2007), Amazing Spider-Man, The (2012), Little Nicky (2000), Mr. Nobody (2009), Sucker Punch (2011), Bulletproof Monk (2003), Where the Wild Things Are (2009), Queen of the Damned (2002), Lady in the Water (2006), Girl Who Leapt Through Time, The (Toki o kakeru shôjo) (2006), Highlander: Endgame (Highlander IV) (2000), Tree of Life, The (2011), Night Watch (Nochnoy dozor) (2004), Daybreakers (2010), Black Knight (2001), Oz the Great and Powerful (2013), Scooby-Doo 2: Monsters Unleashed (2004), Hansel & Gretel: Witch Hunters (2013), Down to Earth (2001), Beasts of the Southern Wild (2012), Dracula Untold (2014), Abraham Lincoln: Vampire Hunter (2012), Ruby Sparks (2012), Troll Hunter, The (Trolljegeren) (2010), Teenage Mutant Ninja Turtles (2014), TMNT (Teenage Mutant Ninja Turtles) (2007), Sausage Party (2016), Priest (2011), Ghostbusters (2016), Crimson Peak (2015), Your Highness (2011), Day Watch (Dnevnoy dozor) (2006), FLCL (2000), Ring, The (2002), Cell, The (2000), Scream 3 (2000), Final Destination (2000), Blade II (2002), Resident Evil (2002), Sweeney Todd: The Demon Barber of Fleet Street (2007), One Hour Photo (2002), Saw II (2005), 1408 (2007), Mothman Prophecies, The (2002), Final Destination 2 (2003), Jeepers Creepers (2001), Silent Hill (2006), Paranormal Activity (2009), 30 Days of Night (2007), Freddy vs. Jason (2003), Cabin Fever (2002), Antichrist (2009), Hills Have Eyes, The (2006), Urban Legends: Final Cut (2000), It Follows (2014), Session 9 (2001), Babadook, The (2014), Skin I Live In, The (La piel que habito) (2011), Jason X (2002), House of 1000 Corpses (2003), What We Do in the Shadows (2014), Purge, The (2013), Slither (2006), Let Me In (2010), Ginger Snaps (2000), Funny Games U.S. (2007), Strangers, The (2008), Omen, The (2006), Triangle (2009), Paranormal Activity 2 (2010), The Witch (2015), Darkness Falls (2003), Halloween (2007), Final Destination, The (Final Destination 4) (Final Destination in 3-D, The) (2009), Halloween: Resurrection (Halloween 8) (2002), High Tension (Haute tension) (Switchblade Romance) (2003), Hostel: Part II (2007), May (2002), Vacancy (2007), Wolf Creek (2005), Fido (2006), Piranha (Piranha 3D) (2010), Oculus (2013), Club Dread (2004), Black Sheep (2006), Paranormal Activity 3 (2011), Insidious: Chapter 2 (2013), Purge: Anarchy, The (2014), House of the Dead, The (2003), Ruins, The (2008), Don't Breathe (2016), Nightmare on Elm Street, A (2010), Dead Silence (2007), John Dies at the End (2012), Diary of the Dead (2007), Reaping, The (2007), V/H/S (2012), Split (2017), Horns (2014), Cursed (2005), Severance (2006), Tremors 3: Back to Perfection (2001), Bone Tomahawk (2015), Midnight Meat Train, The (2008), [REC]² (2009), Hush (2016), The Neon Demon (2016), The Voices (2014), Insidious: Chapter 3 (2015), Last Exorcism, The (2010), Girl Walks Home Alone at Night, A (2014), Housebound (2014), House of the Devil, The (2009), Casshern (2004), Sharknado (2013), Dark Skies (2013), As Above, So Below (2014), Blood: The Last Vampire (2000), Undead (2003), Collector, The (2009), Lights Out (2016), Dead End (2003), Inside (À l'intérieur) (2007), Tusk (2014), Stake Land (2010), Behind the Mask: The Rise of Leslie Vernon (2006), Pulse (Kairo) (2001), Innkeepers, The (2011), Bones (2001), Krampus (2015), Paranormal Activity 4 (2012), Darkest Hour, The (2011), Grave Encounters (2011), Ginger Snaps: Unleashed (2004), Beyond Re-Animator (2003), Don't Be Afraid of the Dark (2010), Wrong Turn 2: Dead End (2007), Scouts Guide to the Zombie Apocalypse (2015), Tournament, The (2009), Spring (2015), Creep (2014), Goodnight Mommy (Ich seh ich seh) (2014), 13 Sins (2014), Wendigo (2001), Rogue (2007), Ginger Snaps Back: The Beginning (2004), Requiem (2006), Maniac (2012), Abandoned, The (2006), Deliver Us from Evil (2014), I Sell the Dead (2008), Life After Beth (2014), Pact, The (2012), Grabbers (2012), Stuck (2007), Hatchet (2006), Honeymoon (2014), Afflicted (2013), We Are What We Are (2013), [REC]³ 3 Génesis (2012), Bay, The (2012), Dance of the Dead (2008), Versus (2000), 2001 Maniacs (2005), Snowtown (Snowtown Murders, The) (2011), Frontière(s) (2007), Absentia (2011), Blair Witch (2016), Children, The (2008), The Green Inferno (2014), Sacrament, The (2013), ABCs of Death, The (2012), Haunter (2013), Cooties (2015), Zombeavers (2014), Woods, The (2006), Universal Soldier: Day of Reckoning (2012), Texas Chainsaw 3D (2013), Southbound (2016), He Never Died (2015), Resolution (2012), Deaths of Ian Stone, The (2007), Woman, The (2011), Last Shift (2014), Stan Helsing (2009), Altered (2006), Cottage, The (2008), Heartless (2009), Last Winter, The (2006), Clown (2014), Dead Snow 2: Red vs. Dead (2014) , Birdemic: Shock and Terror (2010), Horde, The (La Horde) (2009), Crow, The: Wicked Prayer (2005), Kill Command (2016), Escape From Tomorrow (2013), The Gallows (2015), Blood Creek (a.k.a. Town Creek) (2009), Rapture-Palooza (2013), Burrowers, The (2008), Den, The (2013), Bloodsport: The Dark Kumite (1999), Derailed (2002), Belly (1998), Wanted (2008), Black Swan (2010), Source Code (2011), Identity (2003), Eastern Promises (2007), Super 8 (2011), Salt (2010), Manchurian Candidate, The (2004), Eagle Eye (2008), Tinker Tailor Soldier Spy (2011), [REC] (2007), Skeleton Key, The (2005), Book of Shadows: Blair Witch 2 (2000), Final Destination 3 (2006), Joy Ride (2001), Hearts in Atlantis (2001), Salton Sea, The (2002), Non-Stop (2014), Safe House (2012), X-Files: I Want to Believe, The (2008), Buried (2010), Inland Empire (2006), Enemy (2013), Devil (2010), Sinister (2012), Revolver (2005), Mindhunters (2004), The Gift (2015), Inherent Vice (2014), Gossip (2000), Scream 4 (2011), Mirrors (2008), Losers, The (2010), Pontypool (2008), Run All Night (2015), Guest, The (2014), You're Next (2011), Perfect Getaway, A (2009), What Women Want (2000), Pearl Harbor (2001), Meet the Fockers (2004), Wrestler, The (2008), Moonrise Kingdom (2012), Forgetting Sarah Marshall (2008), And Your Mother Too (Y tu mamá también) (2001), Up in the Air (2009), Mr. Deeds (2002), I Heart Huckabees (2004), Adjustment Bureau, The (2011), Save the Last Dance (2001), Bridesmaids (2011), In the Mood For Love (Fa yeung nin wa) (2000), Sweetest Thing, The (2002), Loser (2000), Blue Valentine (2010), Warm Bodies (2013), The Machinist (2004), Island, The (2005), Now You See Me (2013), Red Planet (2000), Death Proof (2007), Layer Cake (2004), Spotlight (2015), Captain Phillips (2013), John Wick (2014), Mission: Impossible - Rogue Nation (2015), Room (2015), Hard Candy (2005), Bridge of Spies (2015), Fracture (2007), Thirteen Ghosts (a.k.a. Thir13en Ghosts) (2001), Rambo (Rambo 4) (2008), John Q (2002), Disturbia (2007), Jack Reacher (2012), Collateral Damage (2002), Conjuring, The (2013), Last Castle, The (2001), Kingdom, The (2007), Ong-Bak: The Thai Warrior (Ong Bak) (2003), Red Eye (2005), Vantage Point (2008), Hitman (2007), Crank: High Voltage (2009), Exorcism of Emily Rose, The (2005), Dracula 2000 (2000), Insidious (2010), Gamer (2009), District 13 (Banlieue 13) (2004), Assault on Precinct 13 (2005), The Lobster (2015), Unstoppable (2010), Fury (2014), Valkyrie (2008), Lone Survivor (2013), Buffalo Soldiers (2001), Brothers (2009), Unbroken (2014), 13 Hours (2016), Red Dawn (2012), '71 (2014), Devils on the Doorstep (Guizi lai le) (2000), 3:10 to Yuma (2007), Hidalgo (2004), Open Range (2003), A Million Ways to Die in the West (2014), The Magnificent Seven (2016), American Outlaws (2001), Good, the Bad, the Weird, The (Joheunnom nabbeunnom isanghannom) (2008), Slow West (2015), Red Hill (2010), Stingray Sam (2009), John Wick: Chapter Two (2017), Digging Up the Marrow (2014), Hatchet II (2010), Frozen (2010), Hatchet III (2013), Lords of Salem, The (2012), 31 (2016), Halloween II (2009), Summer Wars (Samâ wôzu) (2009), The Boy and the Beast (2015), Digimon: The Movie (2000), Die Hard (1988), Alien (1979), Princess Bride, The (1987), Indiana Jones and the Last Crusade (1989), Groundhog Day (1993), Breakfast Club, The (1985), Trading Places (1983), Animal House (1978), Ferris Bueller's Day Off (1986), Cool Hand Luke (1967), Caddyshack (1980), Blazing Saddles (1974), Blade Runner (1982), Airplane! (1980), Hustler, The (1961), There Will Be Blood (2007), Stripes (1981), Road Warrior, The (Mad Max 2) (1981), French Connection, The (1971), Rear Window (1954), Platoon (1986), High Noon (1952), 48 Hrs. (1982), First Blood (Rambo: First Blood) (1982), Wizard of Oz, The (1939), Blue Velvet (1986), Annie Hall (1977), Casablanca (1942), Graduate, The (1967), Scarface (1983), 2001: A Space Odyssey (1968), Upstream Color (2013), Dirty Dozen, The (1967), Bonnie and Clyde (1967), Heat (1995), Running Man, The (1987), Taken 2 (2012), Taken 3 (2015), Da Sweet Blood of Jesus (2014), Chi-Raq (2015), Do the Right Thing (1989), Malcolm X (1992), Summer of Sam (1999), He Got Game (1998), Clockers (1995), Original Kings of Comedy, The (2000), Bamboozled (2000), She's Gotta Have It (1986), Mo' Better Blues (1990), School Daze (1988), Get on the Bus (1996), Rogue One: A Star Wars Story (2016), Star Wars: Episode I - The Phantom Menace (1999), Spaceballs (1987), Ewok Adventure, The (a.k.a. Caravan of Courage: An Ewok Adventure) (1984), Ewoks: The Battle for Endor (1985), Labyrinth (1986), NeverEnding Story, The (1984), Dark Crystal, The (1982), Legend (1985), Time Bandits (1981), Get Out (2017), Apartment, The (1960), Dope (2015), Re-Animator (1985), Return of the Living Dead, The (1985), Day of the Dead (1985), Feast (2005), Bad Taste (1987), Dead Snow (Død snø) (2009), Cemetery Man (Dellamorte Dellamore) (1994), Night of the Living Dead (1968), From Beyond (1986), Signal, The (2007), Creepshow (1982), Splinter (2008), Night of the Creeps (1986), Phantasm (1979), Near Dark (1987), Evil Dead (2013), From Dusk Till Dawn (1996), Tales from the Crypt Presents: Demon Knight (1995), Texas Chainsaw Massacre, The (1974)"

# 모델 입력 구성
test_input = f"Instruction: {instruction}\nInput: {input_text}\nOutput:"


In [ ]:
# 토큰화 및 입력 준비
inputs = tokenizer(test_input, return_tensors="pt", max_length=512, truncation=True)

# 모델 예측
outputs = model.generate(
    inputs.input_ids,
    max_length=50,        # 최대 출력 길이
    num_beams=4,          # Beam Search 사용
    early_stopping=True   # 적절한 시점에 출력 종료
)

# 생성된 텍스트 디코딩
generated_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Recommended Movie:", generated_output)


Recommended Movie: Miracles: Mr. Canton and Lady Rose
